In [1]:
from typing import List, Optional
import pandas as pd
import numpy as np
import sqlite3
import re
import io
import math
import collections
import itertools
import functools
import random
import string
import tqdm
import bisect

conn = sqlite3.connect(":memory:")

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

def read_lc_df(s: str) -> pd.DataFrame:
    temp = pd.read_csv(io.StringIO(s), sep="|", skiprows=2)
    temp = temp.iloc[1:-1, 1:-1]
    temp.columns = temp.columns.map(str.strip)
    temp = temp.map(lambda x: x if type(x) != str else None if x.strip() == 'null' else x.strip())
    return temp

conn.create_function("REGEXP", 2, regexp)

#### Helper for Binary tree problems

In [2]:
class BinaryTreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def to_list(self):
        to_visit = [self]
        visited = []
        while len(to_visit) > 0:
            curr = to_visit.pop(0)
            if curr:
                to_visit.append(curr.left)
                to_visit.append(curr.right)
                visited.append(curr.val)
            else:
                visited.append(curr)

        while visited and not visited[-1]:
            visited.pop()

        return visited

    def __str__(self):
        return str(self.val)

    @staticmethod
    def from_array(nums: list[int|None]):
        '''Create a Tree from a list of nums. Returns the root node.'''
        if len(nums) == 0:
            return None
        elif len(nums) == 1:
            return BinaryTreeNode(nums[0])
        else:
            forest = [BinaryTreeNode(nums[0])]
            parent_idx = -1
            for i in range(1, len(nums)):

                curr = None
                if nums[i] is not None:
                    curr = BinaryTreeNode(nums[i])
                    forest.append(curr)

                if i % 2 == 1:
                    parent_idx += 1
                    forest[parent_idx].left = curr
                else:
                    forest[parent_idx].right = curr

        return forest[0]

#### Helper for Singly Linked lists

In [3]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

    def __str__(self):
        return str(self.val)

    @staticmethod
    def to_singly_linked_list(nums: list[int]):
        root = prev = None
        for n in nums:
            curr = ListNode(n)
            # Init once
            if not root:
                root = curr
            if prev:
                prev.next = curr
            prev = curr

        return root

    def to_list(self) -> list[int]:
        result = []
        curr = self
        while curr:
            result.append(curr.val)
            curr = curr.next
        return result

In [4]:
# Problem Category: Easy
# Link: https://leetcode.com/problems/find-mode-in-binary-search-tree/submissions/1187947888
class Solution:
    def findMode(self, root: BinaryTreeNode) -> List[int]:
        def backtrack(curr: BinaryTreeNode):
            if curr == None:
                return
            else:
                hm[curr.val] = hm.get(curr.val, 0) + 1
                backtrack(curr.left)
                backtrack(curr.right)

        hm: dict[int, int] = dict()
        backtrack(root)
        max_ = max(hm.values())
        result: list[int] = []
        for k, v in hm.items():
            if v == max_:
                result.append(k)
        return result

# Testing the solution
assert Solution().findMode(BinaryTreeNode.from_array([1,None,2,2])) == [2]
assert Solution().findMode(BinaryTreeNode.from_array([0])) == [0]

In [5]:
# Problem Category: Medium
# Link: https://leetcode.com/problems/find-bottom-left-tree-value/submissions/1188894879/
class Solution:
    def findBottomLeftValue(self, root: BinaryTreeNode) -> int:
        next_: list[BinaryTreeNode] = [root]
        while True:
            next_next_: list[BinaryTreeNode] = []
            for node in next_:
                if node.left:
                    next_next_.append(node.left)
                if node.right:
                    next_next_.append(node.right)
            if next_next_:
                next_ = next_next_
            else:
                break
        return next_[0].val

# Testing the solution
assert Solution().findBottomLeftValue(BinaryTreeNode.from_array([2, 1, 3])) == 1
assert Solution().findBottomLeftValue(BinaryTreeNode.from_array([1,2,3,4,None,5,6,None,None,7])) == 7

In [6]:
# Problem Category: Easy
# Link: https://leetcode.com/problems/maximum-odd-binary-number/1190823057
class Solution:
    def maximumOddBinaryNumber(self, s: str) -> str:
        counts = dict(collections.Counter(s))
        zeros = counts.get('0', 0) * '0'
        ones = (counts.get('1', 0) - 1) * '1'
        return ones + zeros + '1'

# Testing the solution
assert Solution().maximumOddBinaryNumber("010") == "001"
assert Solution().maximumOddBinaryNumber("0101") == "1001"

In [7]:
# Problem Category: Easy
# Link: https://leetcode.com/problems/squares-of-a-sorted-array
class Solution:
    def sortedSquares(self, nums: list[int]) -> list[int]:
        N = len(nums)
        i, j = 0, N - 1
        sorted_by_magnitude: list[int] = []
        while i <= j:
            if abs(nums[i]) > abs(nums[j]):
                sorted_by_magnitude.append(nums[i])
                i += 1
            else:
                sorted_by_magnitude.append(nums[j])
                j -= 1
        sorted_by_magnitude.reverse()

        result: list[int] = []
        for num in sorted_by_magnitude:
            result.append(num * num)

        return result

# Testing the solution
assert Solution().sortedSquares([-4,-1,0,3,10]) == [0, 1, 9, 16, 100]
assert Solution().sortedSquares([-7,-4,-1,0,3,10]) == [0, 1, 9, 16, 49, 100]

In [8]:
# Problem Category: Medium
# Link: https://leetcode.com/problems/minimum-length-of-string-after-deleting-similar-ends/submissions/1194814297
class Solution:
    def minimumLength(self, s: str) -> int:
        N = len(s)
        i, j = 0, N - 1
        while i < j:
            while i < N - 1 and s[i + 1] == s[i] and s[i] == s[j]:
                i += 1
            while j > 1 and s[j - 1] == s[j] and s[i] == s[j]:
                j -= 1
            if s[i] == s[j]:
                i += 1
                j -= 1
            else:
                return (j - i) + 1
        else:
            return 1 if i == j else 0

# Testing the solution
assert Solution().minimumLength("ca") == 2
assert Solution().minimumLength("cabaabac") == 0
assert Solution().minimumLength("aabccabba") == 3
assert Solution().minimumLength("bbbbbabbbbccbcbcbccbbabbb") == 1
assert Solution().minimumLength("aaaa") == 0

In [9]:
# Problem Category: Easy
# Link: https://leetcode.com/problems/ransom-note/submissions/1195815128
class Solution:
    def canConstruct(self, ransomNote: str, magazine: str) -> bool:
        cr = collections.Counter(ransomNote)
        cm = collections.Counter(magazine)
        for i in range(ord("a"), ord("z") + 1):
            ch = chr(i)
            if cm.get(ch, 0) < cr.get(ch, 0):
                return False
        else:
            return True

# Testing the solution
assert Solution().canConstruct("aa", "aab") == True
assert Solution().canConstruct("a", "b") == False
assert Solution().canConstruct("aa", "ab") == False

Striver: Pascal's triangle
Problem: Finding nCr in minimal time (part of pascal's triangle)
Find element at position 1st 2nd column in a pascal's triangle
Sol: math.comb(n -1, r - 1)

In [10]:
def nCr(n: int, r: int) -> float:
    num = functools.reduce(lambda x, y: x * y, range(n, n - r, -1))
    den = functools.reduce(lambda x, y: x * y, range(r, 0, -1))
    return num / den

# Testing the solution
for i in range(50):
    i, j = sorted([random.randint(1, 50), random.randint(1, 50)], reverse=True)
    assert math.comb(i, j) == nCr(i, j), f"failed for {i, j}"

In [11]:
# 4Sum
class Solution:
    def fourSum(self, arr: list[int], target: int) -> set[tuple[int]]:
        def update_until(idx: int, update: int, until: int):
            temp_idx = idx
            assert update != 0
            if update > 0:
                while temp_idx < until and arr[temp_idx] == arr[idx]:
                    temp_idx += update
            else:
                while temp_idx > until and arr[temp_idx] == arr[idx]:
                    temp_idx += update
            return temp_idx

        N = len(arr)
        arr.sort()
        result: set[tuple[int]] = set()
        i = 0
        while i < N - 3:
            j = i + 1
            while j < N - 2:
                k, l = j + 1, N - 1
                while k < l:
                    curr = arr[i], arr[j], arr[k], arr[l]
                    if sum(curr) == target:
                        result.add(curr)
                        # Increment k, l
                        k_, l_ = k, l
                        while k_ < l_ and arr[k_] == arr[k] and arr[l_] == arr[l]:
                            k_, l_ = k_ + 1, l_ - 1
                        k, l = k_, l_
                    elif sum(curr) < target:
                        # Increment k
                        k = update_until(k, 1, l)
                    else:
                        # Decrement l
                        l = update_until(l, -1, k)

                # Increment j
                j = update_until(j, 1, N - 2)

            # Increment i
            i = update_until(i, 1, N - 3)

        return result

# Testing the solution
assert Solution().fourSum([1,0,-1,0,-2,2], 0) == {(-2,-1,1,2), (-2,0,0,2), (-1,0,0,1)}
assert Solution().fourSum([2,2,2,2,2], 8) == {(2, 2, 2, 2)}

Leetcode Weekly contest - 17th March

In [12]:
class Solution:
    def isSubstringPresent(self, s: str) -> bool:
        rev_ = "".join(list(reversed(s)))
        N = len(s)
        for i in range(N - 1):
            if s[i:i+2] in rev_:
                return True
        return False

# Testing the solution
assert Solution().isSubstringPresent("leetcode") == True
assert Solution().isSubstringPresent("abcd") == False

zz - z, z, zz (3)
zzz - z, z, z, zz, zz, zzz (6)
zzzz - z, z, z, z; zz, zz, zz, zzz, zzz, zzzz (10)
zzzzz - z, z, z, z, z, zz, zz, zz, zz, zzz, zzz, zzz, zzzz, zzzz, zzzzz (15)

In [13]:
class Solution:
    def countSubstrings(self, s: str, c: str) -> int:
        count = 0
        for ch in s:
            if ch == c:
                count += 1
        return (count * (count + 1)) / 2

# Testing the solution
assert Solution().countSubstrings("abada", "a") == 6
assert Solution().countSubstrings("zzz", "z") == 6

In [14]:
class Solution:
    def minimumDeletions(self, word: str, k: int) -> int:
        freq: list[int] = sorted(collections.Counter(word).values())
        N = len(freq)

        result = math.inf
        sum_ = 0
        for i in range(N):
            sum_ += freq[i - 1] if i > 0 else 0
            curr_ = 0
            for j in range(i + 1, N):
                if freq[j] - freq[i] > k:
                    curr_ += freq[j] - freq[i] - k
            result = min(curr_ + sum_, result)

        return result

# Testing the solution
assert Solution().minimumDeletions("aabcaba", 0) == 3
assert Solution().minimumDeletions("dabdcbdcdcd", 2) == 2
assert Solution().minimumDeletions("aaabaaa", 2) == 1

Leetcode Weekly contest - 17th March
Link: https://leetcode.com/problems/search-in-rotated-sorted-array/submissions/1210464524/

In [15]:
class Solution:
    def search(self, nums: list[int], target: int) -> int:
        N = len(nums)
        low, high = 0, N - 1
        while low <= high:
            mid = (low + high) // 2
            if nums[mid] == target:
                return mid

            # Left portion is sorted
            elif nums[low] <= nums[mid]:
                if nums[low] <= target <= nums[mid]:
                    high = mid - 1
                else:
                    low = mid + 1

            # Right portion is sorted
            else:
                if nums[mid] <= target <= nums[high]:
                    low = mid + 1
                else:
                    high = mid - 1

        # Not found
        else:
            return -1

# Testing the solution
assert Solution().search([4,5,6,7,0,1,2], 0) == 4
assert Solution().search([4,5,6,7,0,1,2], 3) == -1
assert Solution().search([1], 0) == -1

Leetcode weekly contest 24th March, 2024

In [16]:
# Q1 - Easy
class Solution:
    def maximumLengthSubstring(self, s: str) -> int:
        # Brute force
        N = len(s)
        max_ = 0
        for i in range(N - 1):
            counts = {s[i]: 1}
            for j in range(i + 1, N):
                curr_count = counts.get(s[j], 0) + 1
                if curr_count > 2:
                    break
                else:
                    counts[s[j]] = curr_count
                    max_ = max(max_, j - i + 1)
        return max_

# Testing the solution
assert Solution().maximumLengthSubstring("bcbbbcba") == 4
assert Solution().maximumLengthSubstring("aaaa") == 2

In [17]:
# Q2 - Medium
class Solution:
    def minOperations(self, k: int) -> int:
        "Can be optimized with binary search"
        min_ = k - 1
        for i in range(1, k):
            curr = i + math.ceil(k / i) - 2
            if curr < min_:
                min_ = curr

        return min_

# Testing the solution
assert Solution().minOperations(11) == 5
assert Solution().minOperations(1) == 0

In [18]:
# Q3 - Medium (Timeout)
class Solution:
    def mostFrequentIDs(self, nums: list[int], freq: list[int]) -> list[int]:
        "Brute force: O(N^2)"
        N = len(nums)
        counter = dict()
        result: list[int] = []
        for i in range(N):
            counter[nums[i]] = counter.get(nums[i], 0) + freq[i]
            curr_max = max(counter.items(), key=lambda x: x[1])
            result.append(curr_max[1])

        return result

# Testing the solution
assert Solution().mostFrequentIDs([5,5,3], [2,-2,1]) == [2, 0, 1]
assert Solution().mostFrequentIDs([2,3,2,1], [3,2,-3,1]) == [3,3,2,2]

Category: Medium
Video Link: https://leetcode.com/problems/find-the-duplicate-number

In [19]:
class Solution:
    def findDuplicate_(self, nums: list[int]) -> int:
        # Time - O(N log N)
        nums.sort()

        # Time - O(N)
        for i in range(len(nums) -1):
            if nums[i] == nums[i + 1]:
                return nums[i]

        return -1

    def findDuplicate(self, nums: list[int]) -> int:
        def backtrack(start: int) -> None | int:
            next_ = nums[start]
            if next_ == -1:
                return start
            else:
                nums[start] = -1
                return backtrack(next_)

        for i in range(len(nums)):
            if nums[i] == i:
                nums[i] = -1
            else:
                curr = backtrack(nums[i])
                if curr is not None:
                    return curr
        else:
            return -1

# Testing the solution
assert Solution().findDuplicate([1,3,4,2,2]) == 2
assert Solution().findDuplicate([3,1,3,4,2]) == 3
assert Solution().findDuplicate([3,3,3,3,3]) == 3

#### Leetcode Contest: 31st March 2024 (# 391)

In [20]:
# Q1 - Easy
class Solution:
    def sumOfTheDigitsOfHarshadNumber(self, x: int) -> int:
        result = sum(map(int, str(x)))
        if x % result == 0:
            return result
        else:
            return -1

In [21]:
# Q2 - Medium
class Solution:
    def maxBottlesDrunk(self, numBottles: int, numExchange: int) -> int:
        def exchange(empty: int, exchange: int) -> tuple[int, int, int]:
            """
            Given count of empty bottles and start exchange rate,
            returns number of full bottles we can form and the updated exchange rate

            For eg:

            empty = 10, exchange = 1

             E, R, F
            10, 1, 0
            9,  2, 1
            7,  3, 2
            4,  4, 3
            0,  5, 4

            empty = 9, exchange = 1

            E, R, F
            9, 1, 0
            8, 2, 1
            6, 3, 2
            3, 4, 3

            """
            filled = 0
            while empty >= exchange:
                empty -= exchange
                exchange += 1
                filled += 1

            return empty, exchange, filled

        total = empty = 0
        while numBottles:
            total += numBottles
            empty, numBottles = empty + numBottles, 0
            empty, numExchange, numBottles = exchange(empty, numExchange)

        return total

# Testing the solution
assert Solution().maxBottlesDrunk(10, 3) == 13
assert Solution().maxBottlesDrunk(13, 6) == 15

In [22]:
# Q3 - Medium
class Solution:
    def countAlternatingSubarrays(self, nums: list[int]) -> int:
        count, curr, N = 0, 0, len(nums)
        for i in range(N + 1):
            if i == 0 or (i < N and nums[i - 1] != nums[i]):
                curr += 1
            else:
                count += (curr * (curr + 1)) // 2
                curr = 1

        return count

# Testing the solution
assert Solution().countAlternatingSubarrays([1,0,1,0]) == 10
assert Solution().countAlternatingSubarrays( [0,1,1,1]) == 5

Easy Leetcode question: https://leetcode.com/problems/minimum-common-value

In [23]:
def getCommon(nums1: list[int], nums2: list[int]) -> int:
    N1, N2 = len(nums1), len(nums2)
    i = j = 0
    while i < N1 and j < N2:
        if nums1[i] == nums2[j]:
            return nums1[i]
        elif nums1[i] < nums2[j]:
            i += 1
        else:
            j += 1

    return -1

# Testing the solution
assert getCommon([1, 2, 3], [2, 4]) == 2
assert getCommon([1, 2, 3], [4, 5, 6]) == -1
assert getCommon([1, 2, 3], [3]) == 3

Diameter of binary Tree
Problem Link: https://leetcode.com/problems/diameter-of-binary-tree/submissions/1222192232

In [24]:
def diameterOfBinaryTree(root: BinaryTreeNode) -> int:
    """
    1. Perform a post order traversal
    2. Keep updating a global max_ variable as max(max_, left + right) before returning. This is our final result.
    3. At each recursive step, return max(left, right)
    """

    result: int = 0
    def postOrder(curr: BinaryTreeNode) -> int:
        nonlocal result
        if not curr:
            return 0
        else:
            left_max = postOrder(curr.left)
            right_max = postOrder(curr.right)
            result = max(result, left_max + right_max)
            return max(left_max, right_max) + 1

    postOrder(root)
    return result

# Testing the solution
assert diameterOfBinaryTree(BinaryTreeNode.from_array([1, 2, 3, 4, 5])) == 3
assert diameterOfBinaryTree(BinaryTreeNode.from_array([1, 2])) == 1

Easy Leetcode question 
Problem Link: https://leetcode.com/problems/largest-3-same-digit-number-in-string/submissions/1222201215/

In [25]:
def largestGoodInteger(num: str) -> str:
    for n in "9876543210":
        if n * 3 in num:
            return n * 3
    else:
        return ""

# Testing the solution
largestGoodInteger("6777133339")

'777'

Easy Leetcode question
Problem Link: https://leetcode.com/problems/largest-odd-number-in-string/submissions/1222208825

In [26]:
def largestOddNumber(num: str) -> str:
    N = len(num)
    for i in range(N - 1, -1, -1):
        if int(num[i]) % 2 == 1:
            return num[:i + 1]
    else:
        return ""

# Testing the solution
largestOddNumber("354278")

'35427'

Leetcode easy problem: https://leetcode.com/problems/make-the-string-great/submissions/1224126212

In [27]:
def makeGood(s: str) -> str:
    stack: list[str] = []
    for c in s:
        if not stack or c.lower() != stack[-1].lower() or ord(c) == ord(stack[-1]):
            stack.append(c)
        else:
            stack.pop()

    return "".join(stack)

assert makeGood("leeEetcode") == "leetcode"
assert makeGood("abBAcC") == ""

#### Code Gladiators - 2k24

In [28]:
# Q1
def Q1(P: int, X: int, R1: int, N: int, Y: int, R2: int):

    # Get travel time for comparison
    t1 = (P / X) + R1
    t2 = (N / Y) + R2

    if t1 < t2:
        return "NINA"
    elif t1 > t2:
        return "PAUL"
    else:
        return "BOTH"

# Q2:
def Q2(arr: list[int], N: int) -> int:

    def getPair(i: int, target: int) -> int:
        "Returns pair index j, such that arr[i] + arr[j] == target, returns -1 if not possible"
        for j in positions.get(target - arr[i], []):
            if j != i and j not in picked:
                return j
        else:
            return -1

    # Store all the values to a dict for quick reverse lookups
    positions: dict[int, list] = dict()
    for i in range(N):
        indices: list[int] = positions.get(arr[i], [])
        indices.append(i)
        positions[arr[i]] = indices

    # Weights can be between 2 * min_, 2 * max_
    max_count = 0
    for total_weight in range(min(arr) * 2, (max(arr) * 2) + 1):
        count = 0
        picked: set[int] = set()
        for i in range(N):
            j = getPair(i, total_weight) if i not in picked else -1
            if j != -1:
                picked.add(i)
                picked.add(j)
                count += 1

        max_count = max(max_count, count)

    return max_count

# Testing the solution
assert Q2([1, 1, 3, 4, 2, 2], 6) == 2

Problem link: https://leetcode.com/problems/time-needed-to-buy-tickets/submissions/1227477603

In [29]:
# Time: O(N), Space: O(1)
def timeRequiredToBuy(tickets: list[int], k: int) -> int:
    total = 0
    for i in range(len(tickets)):
        if i <= k:
            total += min(tickets[k], tickets[i])
        else:
            total += min(tickets[k] - 1, tickets[i])

    return total

# Testing the Solution
assert timeRequiredToBuy(tickets=[5, 1, 1, 1],  k=0) == 8
assert timeRequiredToBuy(tickets=[2, 3, 2], k=2) == 6

Problem Link: https://leetcode.com/problems/number-of-students-unable-to-eat-lunch/submissions/1227492211

In [30]:
def countStudents(students: list[int], sandwiches: list[int]) -> int:
    stud_pref: dict[int, int] = collections.Counter(students)
    for n in sandwiches:
        if stud_pref.get(n, 0) == 0:
            return stud_pref.get(0, 0) + stud_pref.get(1, 0)
        else:
            stud_pref[n] -= 1
    return 0

# Testing the Solution
assert countStudents([1,1,0,0], [0,1,0,1]) == 0
assert countStudents([1,1,1,0,0,1], [1,0,0,0,1,1]) == 3

All root to leaf paths in a Binary Tree

In [31]:
def allRootToLeaf(root: BinaryTreeNode) -> list[str]:
    """
    1. Do a pre order traversal.
    2. When a leaf node is encountered. Append the path to result.
    3. If not a leaf node, concat to a growing string path.
    """

    paths: list[str] = []
    def preOrder(curr: BinaryTreeNode, traveled: list[str]):
        traveled.append(str(curr.val))

        if not curr.left and not curr.right:
            nonlocal paths
            paths.append(" ".join(traveled))
        if curr.left:
            preOrder(curr.left, traveled)
        if curr.right:
            preOrder(curr.right, traveled)

        traveled.pop()

    preOrder(root, [])
    return paths

# Testing the solution
for ques, ans in (
        ([1, 2, 3, 4, None, 5, 6, None, 7, None, None, None, None, None, None], ["1 3 5", "1 3 6", "1 2 4 7"]),
        ([1, 2, 3, 4, 5, None, None, None, None, None, None], ["1 2 4", "1 2 5", "1 3"]),
    ):
    temp = allRootToLeaf(BinaryTreeNode.from_array(ques))
    for ans_str in ans:
        assert ans_str in temp, f"{ans_str} not found inside result: {temp}"

Problem Link: https://leetcode.com/problems/lowest-common-ancestor-of-a-binary-tree/submissions/1227536364

In [32]:
def lowestCommonAncestor(root: BinaryTreeNode, p: BinaryTreeNode, q: BinaryTreeNode) -> BinaryTreeNode:
    p_path: list[BinaryTreeNode] = []
    q_path: list[BinaryTreeNode] = []
    def getPathsFromRoot(curr: BinaryTreeNode, traveled: list[BinaryTreeNode]):
        nonlocal p_path, q_path
        traveled.append(curr)
        if curr == p:
            p_path = list(traveled)
        if curr == q:
            q_path = list(traveled)
        if curr.left:
            getPathsFromRoot(curr.left, traveled)
        if curr.right:
            getPathsFromRoot(curr.right, traveled)
        traveled.pop()

    getPathsFromRoot(root, [])

    # Find common ancestor
    NP, NQ = len(p_path), len(q_path)
    i = j = 0
    while i < NP and j < NQ and p_path[i] == q_path[j]:
        i, j = i + 1, j + 1

    return p_path[i - 1]

# Testing the solution
temp: BinaryTreeNode = BinaryTreeNode.from_array([3,5,1,6,2,0,8,None,None,7,4])
assert lowestCommonAncestor(temp, p=temp.left, q=temp.left.right.right).val == 5

Problem Link: https://leetcode.com/problems/maximum-width-of-binary-tree/submissions/1228429287

In [33]:
def widthOfBinaryTree(root: BinaryTreeNode) -> int:
    boundaries: dict[int, tuple[int, int]] = dict()
    max_width = 1
    def backtrack(curr: BinaryTreeNode, height: int, width: int):
        nonlocal boundaries, max_width
        if not curr:
            return
        else:
            curr_lr = boundaries.get(height, (2 ** height, 0))
            boundaries[height] = (min(curr_lr[0], width + 1), max(curr_lr[1], width + 1))
            max_width = max(max_width, boundaries[height][1] - boundaries[height][0] + 1)
            backtrack(curr.left, height + 1, width * 2)
            backtrack(curr.right, height + 1, (width * 2) + 1)

    backtrack(root, 0, 0)
    return max_width

# Testing the Solution
assert widthOfBinaryTree(BinaryTreeNode.from_array([1, 3, 2, 5, None, None, 9, 6, None, 7])) == 7

Leetcode Easy: https://leetcode.com/problems/maximum-average-subarray-i/

In [34]:
def findMaxAverage(nums: list[int], k: int) -> float:
    sum_ = sum(nums[:k])
    max_avg = sum_ / k
    for i in range(k, len(nums)):
        sum_ = sum_ + nums[i] - nums[i - k]
        max_avg = max(max_avg, sum_ / k)

    return max_avg

# Testing the solution
assert findMaxAverage([1,12,-5,-6,50,3], 4) == 12.75
assert findMaxAverage([5], 1) == 5

Easy Leetcode Problem Link: https://leetcode.com/problems/average-of-levels-in-binary-tree/submissions/1228468053

In [35]:
def averageOfLevels(root: BinaryTreeNode) -> list[float]:
    result: list[float] = []
    queue: list[BinaryTreeNode] = [root]
    while queue:
        next_: list[BinaryTreeNode] = []
        sum_ = count_ = 0
        for curr in queue:
            sum_, count_ = sum_ + curr.val, count_ + 1
            if curr.left:
                next_.append(curr.left)
            if curr.right:
                next_.append(curr.right)
        result.append(sum_ / count_)
        queue = next_

    return result

# Testing the solution
assert averageOfLevels(BinaryTreeNode.from_array([3,9,20,None,None,15,7])) == [3.00,14.50,11.00]

Problem Link: https://leetcode.com/problems/all-nodes-distance-k-in-binary-tree

In [36]:
def distanceK(root: BinaryTreeNode, target: int, k: int) -> list[int]:
    adl: dict[int, set[int]] = dict()
    def createGraph(curr: BinaryTreeNode):
        nonlocal adl
        if curr:
            if curr.left:
                curr_neighbours = adl.get(curr.val, set())
                curr_neighbours.add(curr.left.val)
                adl[curr.val] = curr_neighbours
                left_neighbours = adl.get(curr.left.val, set())
                left_neighbours.add(curr.val)
                adl[curr.left.val] = left_neighbours
                createGraph(curr.left)
            if curr.right:
                curr_neighbours = adl.get(curr.val, set())
                curr_neighbours.add(curr.right.val)
                adl[curr.val] = curr_neighbours
                right_neighbours = adl.get(curr.right.val, set())
                right_neighbours.add(curr.val)
                adl[curr.right.val] = right_neighbours
                createGraph(curr.right)

    # Create a graph out of the binary Tree
    createGraph(root)

    # Starting from our target, find all nodes at K distance using BFS
    queue: list[int] = [target]
    visited: set[int] = set([target])
    while k > 0:
        next_: list[int] = []
        for curr in queue:
            for neighbour in adl.get(curr, set()):
                if neighbour not in visited:
                    visited.add(neighbour)
                    next_.append(neighbour)
        queue = next_
        k -= 1

    return queue

# Testing the solution
assert sorted(distanceK(BinaryTreeNode.from_array([3,5,1,6,2,0,8,None,None,7,4]), 5, 2)) == [1, 4, 7]
assert sorted(distanceK(BinaryTreeNode.from_array([3,5,1,6,2,0,8,None,None,7,4]), 4, 2)) == [5, 7]

Construct BT from Preorder and Inorder
Problem Link: https://leetcode.com/problems/construct-binary-tree-from-preorder-and-inorder-traversal/submissions/1230345366

In [37]:
def buildTreeFromPreOrder(inorder: list[int], preorder: list[int]) -> BinaryTreeNode|None:
    """
    Preorder: Root, Left, Right
    Inorder: Left, Root, Right

    Root based spliting.
    Use Preorder to determine which of the bunch is the root.
    Use inorder to determine which of the bunch goes to the left half and which right.
    """

    if not inorder:
        return None
    else:
        inorder_set = set(inorder)
        for root_val in preorder:
            if root_val in inorder_set:
                break

        root = BinaryTreeNode(root_val)
        if len(inorder) > 1:
            idx = inorder.index(root.val)
            left_inorder = inorder[:idx]
            root.left = buildTreeFromPreOrder(left_inorder, preorder)
            right_inorder = inorder[idx + 1:]
            root.right = buildTreeFromPreOrder(right_inorder, preorder)

        return root

# Testing the Solution
buildTreeFromPreOrder(inorder=[9,15,20,7], preorder=[3,9,20,15,7]).to_list()

[9, None, 20, 15, 7]

Construct Binary Tree from Inorder and Postorder Traversal
Problem Link: 

In [38]:
def buildTreeFromPostOrder(inorder: list[int], postorder: list[int]) -> BinaryTreeNode|None:
    """
    Inorder: Root, Left, Right
    Postorder: Left, Right, Root

    Same as FromPreOrder, except that we always take the last element also present in inorder as our root.
    Simply reverse postorder and use the same function as before.
    """
    return buildTreeFromPreOrder(inorder=inorder, preorder=postorder[::-1])

# Testing the Solution
buildTreeFromPostOrder(inorder=[3, 2, 4, 1, 5], postorder=[3, 4, 2, 5, 1]).to_list()

[1, 2, 5, 3, 4]

Medium Leetcode: https://leetcode.com/problems/remove-k-digits

In [39]:
# Brute force solution
def removeKdigitsBrute(num: str, k: int) -> str:
    def removeDigit(curr: str) -> str:
        N = len(curr)
        for i in range(N):
            if i == N - 1 or num[i] > num[i + 1]:
                break

        return num[:i] + num[i + 1:]

    while k > 0:
        num = removeDigit(num)
        k -= 1

    num = num.lstrip("0")
    return "0" if not len(num) else num

# Testing the solution
removeKdigitsBrute("1432219", 1)

'132219'

In [40]:
# Optimal solution after trial and errors: https://leetcode.com/problems/remove-k-digits/submissions/1231175827 
def removeKdigitsBetter(num: str, k: int) -> str:
    "Time: O(N), Space: O(N)"
    stack: list[str] = []
    for n in num:
        while stack and k > 0 and int(stack[-1]) > int(n):
            stack.pop()
            k -= 1
        stack.append(n)

    stack = stack[:-k] if k > 0 else stack
    num = "".join(stack).lstrip("0")
    num = num if num else "0"
    return num

# Testing the solution
for _ in tqdm.tqdm(range(1000)):
    N = random.randint(1, 25)
    i = random.randint(1, N)
    str_ = "".join(random.choices("0123456789", k=N))
    assert removeKdigitsBetter(str_, i) == removeKdigitsBrute(str_, i), f"failed for {str_}, {i}"

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:00<00:00, 35321.94it/s]

Leetcode Weekly contest - Saturday (13th Apr)

In [41]:
# Q1 - Easy
def scoreOfString(s: str) -> int:
    N = len(s)
    result = 0
    for i in range(N - 1):
        result += abs(ord(s[i]) - ord(s[i + 1]))
    return result

In [42]:
# Q2 - Medium
def minRectanglesToCoverPoints(points: list[list[int]], w: int) -> int:
    X = sorted(set(map(lambda x: x[0], points)))
    count = 0
    prev = -1
    for x in X:
        if prev == -1 or x - prev > w:
            count += 1
            prev = x

    return count

# Testing the solution
assert minRectanglesToCoverPoints([[0,0],[1,1],[2,2],[3,3],[4,4],[5,5],[6,6]], 2) == 3
assert minRectanglesToCoverPoints([[2,3],[1,2]], 0) == 2

Binary Watch - Easy LC

In [43]:
# https://leetcode.com/problems/binary-watch/submissions/1231377431
def readBinaryWatch(turnedOn: int) -> list[str]:
    def getValue(code: str) -> tuple[int, int]:
        mapping = {
            "H8": (8, 0), "H4": (4, 0),
            "H2": (2, 0), "H1": (1, 0),

            "M32": (0, 32), "M16": (0, 16),
            "M8": (0, 8), "M4": (0, 4),
            "M2": (0, 2), "M1": (0, 1)
        }
        return mapping[code]

    leds: list[str] = ["H8", "H4", "H2", "H1", "M32", "M16", "M8", "M4", "M2", "M1"]
    possbilities = itertools.combinations(leds, r=turnedOn)

    result: list[str] = []
    for possb in possbilities:
        hour_ = min_ = 0
        for led in possb:
            H, M = getValue(led)
            hour_ += H
            min_ += M

        if 0 <= hour_ <= 11 and 0 <= min_ <= 59:
            result.append(f"{hour_}:{str(min_).zfill(2)}")

    return result

# Testing the solution
readBinaryWatch(1)

['8:00',
 '4:00',
 '2:00',
 '1:00',
 '0:32',
 '0:16',
 '0:08',
 '0:04',
 '0:02',
 '0:01']

Valid Perfect Square: https://leetcode.com/problems/valid-perfect-square/submissions/1231389059

In [44]:
def isPerfectSquare(num: int) -> bool:
    low, high = 0, num
    while low <= high:
        mid = (low + high) // 2
        square = mid * mid
        if square == num:
            return True
        elif square < num:
            low = mid + 1
        else:
            high = mid - 1

    return False

# Testing the solution
assert isPerfectSquare(16) == True
assert isPerfectSquare(3) == False

Range sum query: https://leetcode.com/problems/range-sum-query-immutable/submissions/1231398800

In [45]:
class NumArray:
    def __init__(self, nums: list[int]):
        self.nums = nums
        self.cumsum = list(itertools.accumulate(nums, lambda x, y: x + y))

    def sumRange(self, left: int, right: int) -> int:
        result = self.cumsum[right]
        if left > 0:
            result -= self.cumsum[left - 1]
        return result

# Testing the Solution
obj = NumArray([-2, 0, 3, -5, 2, -1])
assert obj.sumRange(0, 2) == 1
assert obj.sumRange(2, 5) == -1

Cows and bulls: https://leetcode.com/problems/bulls-and-cows/submissions/1231417724

In [46]:
def getHint(secret: str, guess: str) -> str:
    N = len(guess)
    secret_freq = collections.Counter(secret)
    guess_freq = collections.Counter(guess)
    cows = bulls = 0
    for i in range(N):
        if guess[i] == secret[i]:
            bulls += 1
            secret_freq[guess[i]] -= 1
            guess_freq[guess[i]] -= 1

    for k in guess_freq:
        cows += min(guess_freq[k], secret_freq.get(k, 0))

    return f"{bulls}A{cows}B"

# Testing the solution
assert getHint("1122", "1222") == "3A0B"

Weekly LC - Sunday: 14th April, 2024

In [47]:
# Q1 - Easy
def findLatestTime(s: str) -> str:
    result = []
    for i in range(5):
        if s[i] == '?':
            if i == 0:
                result.append('0' if s[1].isdigit() and int(s[1]) > 1 else '1')
            elif i == 1:
                result.append('1' if result[0] == '1' else '9')
            elif i == 3:
                result.append('5')
            else:
                result.append('9')
        else:
            result.append(s[i])

    return ''.join(result)

# Testing the solution
findLatestTime("?2:3?")

'02:39'

In [48]:
# Q2 - Medium
def maximumPrimeDifference(nums: list[int]) -> int:
    primes = {2,3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,79,83,89,97}

    N = len(nums)
    for i in range(N):
        if nums[i] in primes:
            break

    for j in range(N - 1, -1, -1):
        if nums[j] in primes:
            break

    return j - i

# Testing the solution
assert maximumPrimeDifference([4,2,9,5,3]) == 3
assert maximumPrimeDifference([4,8,2,8]) == 0

In [49]:
# Q3 - Hard
def findKthSmallestBrute(coins: list[int], k: int) -> int:
    curr = min(coins)
    while True:
        for coin in coins:
            if curr % coin == 0:
                k -= 1
                break

        if k == 0:
            return curr
        else:
            curr += 1

    return -1

# Testing the solution
assert findKthSmallestBrute([3,6,9], 3) == 9
assert findKthSmallestBrute([5,2], 7) == 12

# Q3 - Hard Better (Solved this post the test, proud moment since the acceptance rate was 3%!
# Leetcode Writeup: https://leetcode.com/problems/kth-smallest-amount-with-single-denomination-combination/solutions/5020415/binary-search-solution-python
def findKthSmallest(coins: list[int], k: int) -> int:
    N = len(coins)

    def count(num: int) -> int:
        """
        Given num, find k_ (position after set union and sorting)
        For eg:
            num = 20, coins = [2, 5]
            => [2, 4, 5, 6, 8, 10, 12, 14, 15, 16, 18, 20]
            => 12

        Logic:
            Assume we only had coin 2 available, 20's position would then be 20 // 2 = 10.
            We however have other denominations, so find count for each denomination.
            This way we would introduce duplicates - 10 would be counted twice.
            To avoid this we use formula: count(A | B) = count(A) + count(B) - count(A & B) extended for arbitrary length (A, B, C, ..)
        """
        total, add_ = 0, True

        for i in range(1, N + 1):
            curr = 0
            for comb in itertools.combinations(coins, r=i):
                curr += num // math.lcm(*comb)
            total = total + curr if add_ else total - curr
            add_ = not add_

        return total

    # Use a simple binary search
    # max => lowest denomination * k. The result must be lesser than this.
    low = min(coins)
    high = low * k
    while low <= high:
        mid = (low + high) // 2
        k_ = count(mid)
        if k_ < k:
            low = mid + 1
        else:
            high = mid - 1

    return low

# Testing the solution
for i in tqdm.tqdm(range(1, 100)):
    temp = list(range(11, 26))
    assert findKthSmallest(temp, i) == findKthSmallestBrute(temp, i)

  0%|          | 0/99 [00:00<?, ?it/s]

  2%|▏         | 2/99 [00:00<00:05, 16.21it/s]

  4%|▍         | 4/99 [00:00<00:09, 10.14it/s]

  6%|▌         | 6/99 [00:00<00:11,  8.31it/s]

  7%|▋         | 7/99 [00:00<00:11,  7.86it/s]

...

 97%|█████████▋| 96/99 [00:20<00:00,  4.08it/s]

 98%|█████████▊| 97/99 [00:20<00:00,  4.08it/s]

 99%|█████████▉| 98/99 [00:20<00:00,  4.08it/s]

100%|██████████| 99/99 [00:20<00:00,  4.08it/s]

100%|██████████| 99/99 [00:20<00:00,  4.72it/s]

Sum of left leaves
Problem Link: https://leetcode.com/problems/sum-of-left-leaves/submissions/1232098618

In [50]:
def sumOfLeftLeaves(root: BinaryTreeNode, left: bool = True) -> int:
    if not root:
        return 0
    elif not root.left and not root.right:
        return root.val if left else 0
    else:
        return sumOfLeftLeaves(root.left, True) + sumOfLeftLeaves(root.right, False)

Minimum element in BST

In [51]:
def minValue(root: BinaryTreeNode) -> int:
    "Time Complexity: O(N)"
    curr: BinaryTreeNode = root
    while curr and curr.left:
        curr = curr.left

    return curr.val if curr else -1

Floor from BST

In [52]:
# Read input as specified in the question
# target, *nums = list(map(lambda x: int(x) if x != '-1' else None, input().split()))
target, nums = 11, [5, 2, 9, 1, 3, 6, 10]
k: float = -1
queue: list[BinaryTreeNode] = []
for num in nums:
    curr = BinaryTreeNode(num) if num else None
    if curr:
        queue.append(curr)
    if k == -1:
        k += 1
    elif int(k) == k:
        queue[int(k)].left = curr
        k += 0.5
    else:
        queue[int(k)].right = curr
        k += 0.5

queue[0].to_list()

# Find the floor of the number
curr, floor = queue[0], -1
while curr:
    if curr.val <= target:
        floor = curr.val
        curr = curr.right
    else:
        curr = curr.left

print(floor)

10


Insert into a BST

In [53]:
def insertIntoBST(root: BinaryTreeNode, val: int) -> BinaryTreeNode:
    if not root:
        root = BinaryTreeNode(val)
    elif root.val < val:
        root.right = insertIntoBST(root.right, val)
    else:
        root.left = insertIntoBST(root.left, val)

    return root

# Testing the solution
assert insertIntoBST(BinaryTreeNode.from_array([4,2,7,1,3]), 5).to_list() == [4,2,7,1,3,5]

LCA in a BST of two nodes: https://leetcode.com/problems/lowest-common-ancestor-of-a-binary-search-tree/submissions/1234964550

In [ ]:
def lowestCommonAncestor(root: BinaryTreeNode, p: BinaryTreeNode, q: BinaryTreeNode) -> BinaryTreeNode:
    """
    If root equals p/q or if p < root < q that is our LCA. Otherwise traverse left of right.
    Time: O(log N), Space: O(1)
    """

    low = p if p.val < q.val else q
    high = q if p.val < q.val else p

    if root.val == low.val or root.val == high.val or low.val < root.val < high.val:
        return root
    elif root.val > p.val and root.val > q.val:
        return lowestCommonAncestor(root.left, p, q)
    else:
        return lowestCommonAncestor(root.right, p, q)

# Testing the solution
temp: BinaryTreeNode = BinaryTreeNode.from_array([6,2,8,0,4,7,9,None,None,3,5])
assert lowestCommonAncestor(temp, p=temp.left, q=temp.right).val == 6

Smallest string starting from leaf: https://leetcode.com/problems/smallest-string-starting-from-leaf/submissions/1235018957

In [ ]:
def smallestFromLeaf(root: BinaryTreeNode) -> str:
    result: str = chr(ord('z') + 1)
    def backtrack(curr: BinaryTreeNode, acc: str = ""):
        if curr:
            acc_ = chr(curr.val + ord('a')) + acc
            if not curr.left and not curr.right:
                nonlocal result
                result = min(result, acc_)
            else:
                backtrack(curr.left, acc_)
                backtrack(curr.right, acc_)

    backtrack(root)
    return result

# Testing the solution
assert smallestFromLeaf(BinaryTreeNode.from_array([25,1,3,1,3,0,2])) == "adz"
assert smallestFromLeaf(BinaryTreeNode.from_array([2,2,1,None,1,0,None,0])) == "abc"
assert smallestFromLeaf(BinaryTreeNode.from_array([4,0,1,1])) == "bae"
assert smallestFromLeaf(BinaryTreeNode.from_array([25,1,None,0,0,1,None,None,None,0])) == "ababz"